In [1]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... - \ done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=5c3e70821fa2557143b1017e0e4a3f71c9a23a5d7895d46d6f52f86de70994a4
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [2]:
import os
import torch
import random
import urllib3
import numpy as np
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
from torchvision import transforms
from torch.nn import functional as F
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from efficientnet_pytorch import EfficientNet
from pytorch_lightning.callbacks import ModelCheckpoint

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# 데이터 경로
data_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/'
# 훈련, 검증, 테스트 경로 설정
train_path = data_path + 'train/'
valid_path = data_path + 'val/'
test_path = data_path + 'test/'

In [5]:
# 훈련 데이터용 변환기
transform_train = transforms.Compose([
    transforms.Resize((250, 250)), # 이미지 크기 조정
    transforms.CenterCrop(180), # 중앙 이미지 확대
    transforms.RandomHorizontalFlip(0.5), # 좌우 대칭
    transforms.RandomVerticalFlip(0.2), # 상하 대칭
    transforms.RandomRotation(20), # 이미지 회전
    transforms.ToTensor(), # 텐서 객체로 변환
    transforms.Normalize((0.485, 0.456, 0.406),
                         (0.229, 0.224, 0.225)) # 정규화
])

# 테스트 데이터용 변환기
transform_test = transforms.Compose([
    transforms.Resize((250, 250)),
    transforms.CenterCrop(180),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),
                         (0.229, 0.224, 0.225))
])

In [6]:
datasets_train = ImageFolder(root=train_path, transform=transform_train)
datasets_valid = ImageFolder(root=valid_path, transform=transform_test)

In [7]:
batch_size = 8

loader_train = DataLoader(dataset=datasets_train, batch_size=batch_size, shuffle=True)
loader_valid = DataLoader(dataset=datasets_valid, batch_size=batch_size, shuffle=False)

In [8]:
# train 시킬떄 순서 training_step -> validataion_step -> validation_epoch_end
class Baseline_EfficientNet(pl.LightningModule):
    def __init__(self, num_classes):
        super().__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=2).to(device) 
    
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = nn.CrossEntropyLoss()(y_hat, y)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x,y = batch
        y_hat = self.forward(x)
        loss = nn.CrossEntropyLoss()(y_hat, y)
        return loss

    def configure_optimizers(self):    
        return torch.optim.Adam(self.model.parameters(), lr=0.01)

    def train_dataloader(self):
        return loader_train

    def val_dataloader(self):
        return loader_valid

In [9]:
model = Baseline_EfficientNet(num_classes=2)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:00<00:00, 154MB/s]


Loaded pretrained weights for efficientnet-b0


In [10]:
trainer = Trainer(devices=1, accelerator="gpu", max_epochs=10)
trainer.fit(model)

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [11]:
datasets_test = ImageFolder(root=test_path, transform=transform_test)

loader_test = DataLoader(dataset=datasets_test, batch_size=batch_size, shuffle=False)

In [12]:
model.to(device)

Baseline_EfficientNet(
  (model): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d((0, 1, 0, 1))
    )
    (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
          (static_padding): ZeroPad2d((1, 1, 1, 1))
        )
        (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          32, 8, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          8, 32, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_project_conv): Conv2dStaticSam

In [13]:
trues_list = []
preds_list = []

model.eval()
for images, labels in loader_test:
    images = images.to(device)
    labels = labels.to(device)
    
    outputs = model(images)
    
    preds = torch.max(outputs.cpu(), dim=1)[1].numpy() # 에측값
    trues = labels.cpu().numpy() # 실젯값
    
    preds_list.extend(preds)
    trues_list.extend(trues)

In [14]:
from sklearn.metrics import accuracy_score # 정확도 계산 함수
from sklearn.metrics import recall_score # 재현율 계산 함수
from sklearn.metrics import f1_score # F1 점수 계산 함수

print(f'##### 최종 예측 결과 평가 점수 #####')
print(f'정확도 : {accuracy_score(trues_list, preds_list):.4f}')
print(f'재현율 : {recall_score(trues_list, preds_list):.4f}')
print(f'F1 점수 :{f1_score(trues_list, preds_list):.4f}')

##### 최종 예측 결과 평가 점수 #####
정확도 : 0.8686
재현율 : 0.9564
F1 점수 :0.9010
